In [ ]:
import os
import json
import pandas as pd
import psycopg2
import streamlit as st 
import subprocess
subprocess.run(['git', 'clone', 'https://github.com/PhonePe/pulse.git'])
import plotly.express as px
import plotly.colors as pc
import matplotlib.pyplot as plt
import requests
import geopandas as gpd
import base64

mydb = psycopg2.connect(host = 'localhost',
                       user = 'postgres',
                       port = '5432',
                       database ='phonepe',
                       password = 'Liverpool08#')
cursor = mydb.cursor()

url = "https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson"
response = requests.get(url)
india_geojson = json.loads(response.content)  

### arrrigated transaction

def aggregated_transaction_map_with_allstate_year_quarter(cursor, year=None, quarter=None, column = None):
    
    query = """
    SELECT states, 
           SUM(transaction_count) AS transaction_count, 
           SUM(transaction_amount) AS transaction_amount
    FROM aggregated_transaction
    WHERE 1=1
    """
    params = {}
    if year:
        query += " AND year = %(year)s"
        params["year"] = year
    if quarter:
        query += " AND quarter = %(quarter)s"
        params["quarter"] = quarter
    query += " GROUP BY states;"

    cursor.execute(query, params)
    result = cursor.fetchall()

    df = pd.DataFrame(result, columns=["State", "Transaction Count", "Transaction Amount"])
    df["Transaction Count"] = df["Transaction Count"].astype("int64")
    df["Transaction Amount"] = df["Transaction Amount"].astype("int64")

    title_suffix = f" over {year}" if year else ""
    title_suffix2 = f"Q{quarter}" if quarter else ""

    fig = px.choropleth(
        df,
        geojson=india_geojson,
        featureidkey="properties.ST_NM",
        locations="State",
        color=column,  
        color_continuous_scale="Viridis",  
        range_color=[df[column].min(), df[column].max()],  
        hover_name = "State",
        title= f"All India {column} {title_suffix} {title_suffix2}",
        fitbounds= "locations",width =900, height= 600,
    )
    
    fig.update_geos(visible=False)
    st.plotly_chart(fig, use_container_width=True)
    return 



def aggregated_transaction_with_state_year_quarter(cursor, year=None, quarter=None):
    query = """
    SELECT states, 
           SUM(transaction_count) AS transaction_count, 
           SUM(transaction_amount) AS transaction_amount
    FROM aggregated_transaction
    WHERE 1=1
    """
    params = {}
    if year:
        query += " AND year = %(year)s"
        params["year"] = year
    if quarter:
        query += " AND quarter = %(quarter)s"
        params["quarter"] = quarter
    query += " GROUP BY states;"

    cursor.execute(query, params)
    result = cursor.fetchall()

    df = pd.DataFrame(result, columns=["State", "Transaction Count", "Transaction Amount"])
   
    title_suffix = f" in {year}" if year else ""
    title_suffix_1  = f" Q {quarter}" if year else ""

    df_long = df.melt(id_vars=['State'], 
                      value_vars=['Transaction Count', 'Transaction Amount'], 
                      var_name='Select', 
                      value_name='Value')

    fig_stacked = px.bar(
        df_long,
        x='State',
        y='Value',
        color='Select',
        title=f"Transactions by all States {title_suffix} {title_suffix_1}",
        barmode='stack',
        color_discrete_map={
            "Transaction Count": "blue",
            "Transaction Amount": "green"
        }
    )

    fig_stacked.update_layout(
        title={
            'text': f"Transactions by all States in India {title_suffix} {title_suffix_1}",  
            'font': {'color': 'purple', 'size': 22},  
            'x': 0.5  
        },
        plot_bgcolor='rgba(230, 230, 230, 0.5)',  
        paper_bgcolor='lightyellow',  
        xaxis_title={
            'text': 'States',
            'font': {'color': 'blue', 'size': 18}  
        },
        yaxis_title={
            'text': 'Transaction Value',
            'font': {'color': 'green', 'size': 18}  
        },
        font=dict(
            family="Arial, sans-serif",
            size=14,
            color="darkblue"  
        )
    )

    st.plotly_chart(fig_stacked, use_container_width=True)
    return 


def aggregated_transaction_transaction_type_with_state_year_quarter(cursor, year=None, state=None, quarter=None):
    query = """SELECT states, transaction_type, 
                      SUM(transaction_count) AS transaction_count, 
                      SUM(transaction_amount) AS transaction_amount
               FROM aggregated_transaction
               WHERE 1=1"""
    
    params = {}
    if state:
        query += " AND states = %(state)s"
        params["state"] = state
    if year:
        query += " AND year = %(year)s"
        params["year"] = year
    if quarter:
        query += " AND quarter = %(quarter)s"
        params["quarter"] = quarter
    query += " GROUP BY states, transaction_type;"

    cursor.execute(query, params)
    result = cursor.fetchall()

    df = pd.DataFrame(result, columns=["states", "transaction_type", "Transaction Count", "Transaction Amount"])
    df["Transaction Count"] = df["Transaction Count"].astype("int64")
    df["Transaction Amount"] = df["Transaction Amount"].astype("int64")
    
    custom_colors = px.colors.qualitative.Set3 
    fig1 = px.pie(
        df,
        names="transaction_type",
        values="Transaction Count",  
        title=f"Transaction Count Distribution by Type in {state if state else 'All States'}",
        hole=0.3,
        color="transaction_type",
        color_discrete_sequence=custom_colors
    )
    fig2 = px.pie(
        df,
        names="transaction_type",
        values="Transaction Amount", 
        title=f"Transaction Amount Distribution by Type in {state if state else 'All States'}",
        hole=0.3,
        color="transaction_type",
        color_discrete_sequence=custom_colors
    )
    col1, col2 = st.columns(2)
    with col1:
        st.plotly_chart(fig1, use_container_width=True)
    with col2:
        st.plotly_chart(fig2, use_container_width=True)  

    return  



### agg user



def aggregated_user_brands_state_year_quarter(cursor, year=None,state =None, quarter=None):
    query = """
    SELECT States,year,quarter,brands,percentage
    FROM aggrigated_user
    WHERE 1=1
    """
    params = {}
    if state:
        query += " AND states = %(state)s"
        params["state"] = state
    if year:
        query += " AND year = %(year)s"
        params["year"] = year
    if quarter:
        query += " AND quarter = %(quarter)s"
        params["quarter"] = quarter

    cursor.execute(query, params)
    result = cursor.fetchall()

    df = pd.DataFrame(result, columns=["State","year","quarter","brands", "percentage"])
    title_suffix = f" over {year}" if year else ""
    title_suffix1 = f" at {state}" if state else ""
    title_suffix2 = f"Q{quarter}" if quarter else ""
    
    custom_colors = px.colors.qualitative.Set3 
    fig = px.pie(
        df,
        names="brands",
        values="percentage", 
        title=f"Brands Distribution {title_suffix1} {title_suffix} {title_suffix2}",
        hole=0.3,
        color="brands",
        color_discrete_sequence=custom_colors
    )
    st.plotly_chart(fig, use_container_width=True)

    return 

### map trans

def map_transaction_district_with_state_year_quarter(cursor, year=None, states=None, quarter=None):
    query = """SELECT states, district, 
                      SUM(transaction_count) AS transaction_count, 
                      SUM(transaction_amount) AS transaction_amount
               FROM df_map_trans
               WHERE 1=1"""
    
    params = {}
    if states:
        query += " AND states = %(states)s"
        params["states"] = states
    if year:
        query += " AND year = %(year)s"
        params["year"] = year
    if quarter:
        query += " AND quarter = %(quarter)s"
        params["quarter"] = quarter
    query += " GROUP BY states, district;"

    cursor.execute(query, params)
    result = cursor.fetchall()

    df = pd.DataFrame(result, columns=["states", "district", "Transaction Count", "Transaction Amount"])
    df["Transaction Count"] = df["Transaction Count"].astype("int64")
    df["Transaction Amount"] = df["Transaction Amount"].astype("int64")
    
    title_suffix = f" in {year}" if year else ""
    title_suffix_1  = f" Q {quarter}" if year else ""

    df_long = df.melt(id_vars=['district'], 
                      value_vars=['Transaction Count', 'Transaction Amount'], 
                      var_name='Select', 
                      value_name='Value')

    fig_stacked = px.bar(
        df_long,
        x='district',
        y='Value',
        color='Select',
        title=f"{states} District  {title_suffix} {title_suffix_1}",
        barmode='stack',
        color_discrete_map={
            "Transaction Count": "blue",
            "Transaction Amount": "green"
        }
    )

    fig_stacked.update_layout(
        title={
            'text': f"{states} District  {title_suffix} {title_suffix_1}",  
            'font': {'color': 'purple', 'size': 22},  
            'x': 0.5  
        },
        plot_bgcolor='rgba(230, 230, 230, 0.5)',  
        paper_bgcolor='lightyellow',  
        xaxis_title={
            'text': 'Districts',
            'font': {'color': 'blue', 'size': 18}  
        },
        yaxis_title={
            'text': 'Transaction',
            'font': {'color': 'green', 'size': 18}  
        },
        font=dict(
            family="Arial, sans-serif",
            size=14,
            color="darkblue"  
        )
    )
    st.plotly_chart(fig_stacked, use_container_width=True)

    return

   


### map user

def map_user_map_with_allstate_year_quarter(cursor, year=None, quarter=None, column = None):
    query = """
    SELECT states, 
           SUM(registeredusers) AS "Registered users", 
           SUM(appopens) AS "App opens"
    FROM df_map_user
    WHERE 1=1
    """
    params = {}
    if year:
        query += " AND year = %(year)s"
        params["year"] = year
    if quarter:
        query += " AND quarter = %(quarter)s"
        params["quarter"] = quarter
    query += " GROUP BY states;"

    cursor.execute(query, params)
    result = cursor.fetchall()

    df = pd.DataFrame(result, columns=["State", "Registered users", "App opens"])
    df["Registered users"] = df["Registered users"].astype("int64")
    df["App opens"] = df["App opens"].astype("int64")

    title_suffix = f" over {year}" if year else ""
    title_suffix_1  = f" Q {quarter}" if year else ""

    fig = px.choropleth(
        df,
        geojson=india_geojson,
        featureidkey="properties.ST_NM",
        locations="State",
        color=column,  
        color_continuous_scale="Viridis",  
        range_color=[df[column].min(), df[column].max()],  
        hover_name = "State",
        title=f" Users in India {title_suffix} {title_suffix_1}",
        fitbounds= "locations",width =900, height= 600,
        )
    
    fig.update_geos(visible=False)
    st.plotly_chart(fig, use_container_width=True)
    return 





def map_user_with_state_year_quarter(cursor, year=None, quarter=None):
    query = """
    SELECT states, 
           SUM(registeredusers) AS "Registered users", 
           SUM(appopens) AS "App opens"
    FROM df_map_user
    WHERE 1=1
    """
    params = {}
    if year:
        query += " AND year = %(year)s"
        params["year"] = year
    if quarter:
        query += " AND quarter = %(quarter)s"
        params["quarter"] = quarter

    query += " GROUP BY states;"

    cursor.execute(query, params)
    result = cursor.fetchall()

    df = pd.DataFrame(result, columns=["State", "Registered users", "App opens"])

   
    title_suffix = f" in {year}" if year else ""
    title_suffix_1  = f" Q {quarter}" if year else ""

    df_long = df.melt(id_vars=['State'], 
                      value_vars=['Registered users', 'App opens'], 
                      var_name='Select', 
                      value_name='Value')

    fig_stacked = px.bar(
        df_long,
        x='State',
        y='Value',
        color='Select',
        title=f"Users by State{title_suffix} {title_suffix_1}",
        barmode='stack',
        color_discrete_map={
            "Registered users": "blue",
            "App opens": "green"
        }
    )

    fig_stacked.update_layout(
        title={
            'text': f"Users by State{title_suffix} {title_suffix_1}",  
            'font': {'color': 'purple', 'size': 22},  
            'x': 0.5  
        },
        plot_bgcolor='rgba(230, 230, 230, 0.5)',  
        paper_bgcolor='lightyellow',  
        xaxis_title={
            'text': 'States',
            'font': {'color': 'blue', 'size': 18}  
        },
        yaxis_title={
            'text': 'Users',
            'font': {'color': 'green', 'size': 18}  
        },
        font=dict(
            family="Arial, sans-serif",
            size=14,
            color="darkblue"  
        )
    )

    st.plotly_chart(fig_stacked, use_container_width=True)
    return 


### top_trans

def top_transaction_pincodes_with_state_year_quarter(cursor, year=None, states=None, quarter=None):
    query = """SELECT pincodes, 
                      SUM(transaction_count) AS transaction_count, 
                      SUM(transaction_amount) AS transaction_amount
               FROM df_top_trans
               WHERE 1=1"""
    
    params = {}
    if states:
        query += " AND states = %(states)s"
        params["states"] = states
    if year:
        query += " AND year = %(year)s"
        params["year"] = year
    if quarter:
        query += " AND quarter = %(quarter)s"
        params["quarter"] = quarter
    query += " GROUP BY states, pincodes;"

    cursor.execute(query, params)
    result = cursor.fetchall()

    df = pd.DataFrame(result, columns=["Pincodes", "Transaction Count", "Transaction Amount"])
    df["Transaction Count"] = df["Transaction Count"].astype("int64")
    df["Transaction Amount"] = df["Transaction Amount"].astype("int64")
    df["Pincodes"] = df["Pincodes"].astype(str)  
    st.dataframe(df)
    return

   


### top user

def top_user_pincodes_with_state_year_quarter(cursor, year=None, states=None, quarter=None):
    query = """SELECT pincodes, 
                      SUM(registeredusers) AS "Registered users" 
               FROM df_top_user
               WHERE 1=1"""
    
    params = {}
    if states:
        query += " AND states = %(states)s"
        params["states"] = states
    if year:
        query += " AND year = %(year)s"
        params["year"] = year
    if quarter:
        query += " AND quarter = %(quarter)s"
        params["quarter"] = quarter
    query += " GROUP BY states, pincodes;"

    cursor.execute(query, params)
    result = cursor.fetchall()

    df = pd.DataFrame(result, columns=["pincodes", "Registered users"])
    df["Registered users"] = df["Registered users"].astype("int64")
    df["pincodes"] = df["pincodes"].astype(str)
    st.dataframe(df)
    return 

   


### time series 

### agg trans

def quarterly_transaction_data(year, states):
    query = f"""
        SELECT states, year, quarter, 
               SUM(transaction_count) AS "Transaction count", 
               SUM(transaction_amount) AS "Transaction amount"
        FROM aggregated_transaction
        WHERE year = {year} AND states = '{states}'
        GROUP BY states, year, quarter
        ORDER BY quarter ASC;
    """

 
    cursor.execute(query)
    result = cursor.fetchall()

    df = pd.DataFrame(result, columns=["states", "year", "quarter", "Transaction Count", "Transaction Amount"])




    df_long = df.melt(id_vars=['quarter'], 
                      value_vars=['Transaction Count', 'Transaction Amount'], 
                      var_name='Select', 
                      value_name='Value')
    
    fig_line = px.line(
        df_long,
        x='quarter',
        y='Value',
        color='Select',
        title=f"Quarterly analysis of {states} {year}",
        markers=True,  
        color_discrete_map={
            "Transaction Amount": "blue",
            "Transaction Count": "green"
        }
    )
    
    fig_line.update_layout(
        title={
            'text': f"Quarterly analysis of {states} {year}",  
            'font': {'color': 'purple', 'size': 22},  
            'x': 0.5  
        },
        plot_bgcolor='rgba(230, 230, 230, 0.5)',  
        paper_bgcolor='lightyellow',  
        xaxis_title={
            'text': 'Quarter',
            'font': {'color': 'blue', 'size': 18}  
        },
        yaxis_title={
            'text': 'Transaction',
            'font': {'color': 'green', 'size': 18}  
        },
        font=dict(
            family="Arial, sans-serif",
            size=14,
            color="darkblue"  
        ),
        xaxis=dict(
            tickmode='array',  
            tickvals=[1, 2, 3, 4],  
            ticktext=['1', '2', '3', '4']
        )
    )
    
    st.plotly_chart(fig_line, use_container_width=True)

    return

def yearly_transaction_data(states):
    query = """
        SELECT states, year, 
               SUM(transaction_count) AS "Transaction Count", 
               SUM(transaction_amount) AS "Transaction Amount"
        FROM aggregated_transaction
        WHERE states = %s
        GROUP BY states, year
        ORDER BY year ASC;
    """

    cursor.execute(query, (states,))  
    result = cursor.fetchall()

    df = pd.DataFrame(result, columns=["states", "year", "Transaction Count", "Transaction Amount"])

    df_long = df.melt(id_vars=['year'], 
                      value_vars=['Transaction Count', 'Transaction Amount'], 
                      var_name='Select', 
                      value_name='Value')
    
    fig_line = px.line(
        df_long,
        x='year',
        y='Value',
        color='Select',
        title=f"Yearly Analysis of {states}",
        markers=True,  
        color_discrete_map={
            "Transaction Amount": "blue",
            "Transaction Count": "green"
        }
    )
    
    fig_line.update_layout(
        title={'text': f"Yearly Analysis of {states}", 'font': {'color': 'purple', 'size': 22}, 'x': 0.5},
        plot_bgcolor='rgba(230, 230, 230, 0.5)',  
        paper_bgcolor='lightyellow',  
        xaxis_title={'text': 'Years', 'font': {'color': 'blue', 'size': 18}},  
        yaxis_title={'text': 'Transaction', 'font': {'color': 'green', 'size': 18}},  
        font=dict(family="Arial, sans-serif", size=14, color="darkblue"),
        xaxis=dict(
            tickmode='array',  
            tickvals=[2018, 2019, 2020, 2021, 2022, 2023],  
            ticktext=['2018', '2019', '2020', '2021', '2022', '2023']
        )
    )
    
    st.plotly_chart(fig_line, use_container_width=True)
    return  

### aggrigated user

def quarterly_agg_user_data(year, brands):
    query = f"""
        SELECT brands, year, quarter, 
               SUM(transaction_count) AS "Transaction count" 
               
        FROM aggrigated_user
        WHERE year = {year} AND brands = '{brands}'
        GROUP BY brands, year, quarter
        ORDER BY quarter ASC;
    """
    cursor.execute(query)
    result = cursor.fetchall()

    df = pd.DataFrame(result, columns=["brands", "year", "quarter", "Transaction Count"])
    df_long = df.melt(id_vars=['quarter'], 
                      value_vars=['Transaction Count'], 
                      var_name='Select', 
                      value_name='Value')
    
    fig_line = px.line(
        df_long,
        x='quarter',
        y='Value',
        color='Select',
        title=f"Quarterly analysis of {brands} {year}",
        markers=True,  
        color_discrete_map={
           
            "Transaction Count": "green"
        }
    )
    
    fig_line.update_layout(
        title={
            'text': f"Quarterly analysis of {brands} {year}",  
            'font': {'color': 'purple', 'size': 22},  
            'x': 0.5  
        },
        plot_bgcolor='rgba(230, 230, 230, 0.5)',  
        paper_bgcolor='lightyellow',  
        xaxis_title={
            'text': 'Quarter',
            'font': {'color': 'blue', 'size': 18}  
        },
        yaxis_title={
            'text': 'Transaction',
            'font': {'color': 'green', 'size': 18}  
        },
        font=dict(
            family="Arial, sans-serif",
            size=14,
            color="darkblue"  
        ),
        xaxis=dict(
            tickmode='array',  
            tickvals=[1, 2, 3, 4],  
            ticktext=['1', '2', '3', '4']
        )
    )
    
    st.plotly_chart(fig_line, use_container_width=True)
    return


def yearly_aggrigated_user_data(brands):
    query = """
        SELECT brands, year, 
               SUM(transaction_count) AS "Transaction Count" 
        FROM aggrigated_user
        WHERE brands = %s
        GROUP BY brands, year
        ORDER BY year ASC;
    """
    
    cursor.execute(query, (brands,))  
    result = cursor.fetchall()

    df = pd.DataFrame(result, columns=["brands", "year", "Transaction Count"])

    df_long = df.melt(id_vars=['year'], 
                      value_vars=['Transaction Count'], 
                      var_name='Select', 
                      value_name='Value')

    fig_line = px.line(
        df_long,
        x='year',
        y='Value',
        color='Select',
        title=f"Yearly Analysis of {brands}",
        markers=True,  
        color_discrete_map={"Transaction Count": "green"}
    )
    
    fig_line.update_layout(
        title={'text': f"Yearly Analysis of {brands}", 'font': {'color': 'purple', 'size': 22}, 'x': 0.5},
        plot_bgcolor='rgba(230, 230, 230, 0.5)',  
        paper_bgcolor='lightyellow',  
        xaxis_title={'text': 'Years', 'font': {'color': 'blue', 'size': 18}},  
        yaxis_title={'text': 'Transaction', 'font': {'color': 'green', 'size': 18}},  
        font=dict(family="Arial, sans-serif", size=14, color="darkblue"),
        xaxis=dict(
            tickmode='array',  
            tickvals=[2018, 2019, 2020, 2021, 2022, 2023],  
            ticktext=['2018', '2019', '2020', '2021', '2022', '2023']
        )
    )
    
    st.plotly_chart(fig_line, use_container_width=True)
    return 

### map trans

def quarterly_map_trans_district_data(states, year, district):
    query = """
        SELECT states, district, year, quarter,
               transaction_count as "Transaction count",
               transaction_amount as "Transaction amount"
               
        FROM df_map_trans
        WHERE states = %s AND year = %s AND district = %s
    """
    cursor.execute(query, (states, year, district))
    result = cursor.fetchall()

    df = pd.DataFrame(result, columns=["states", "district", "year", "quarter", "Transaction Count", "Transaction amount"])

    df_long = df.melt(id_vars=['quarter'], 
                      value_vars=['Transaction Count',  "Transaction amount"], 
                      var_name='Select', 
                      value_name='Value')
    
    fig_line = px.line(
        df_long,
        x='quarter',
        y='Value',
        color='Select',
        title=f"Quarterly analysis of {district} {year}",
        markers=True,  
        color_discrete_map={
            "Transaction amount": "blue",
            "Transaction Count": "green"
        }
    )
    
    fig_line.update_layout(
        title={
            'text': f"Quarterly analysis of {district} {year}",  
            'font': {'color': 'purple', 'size': 22},  
            'x': 0.5  
        },
        plot_bgcolor='rgba(230, 230, 230, 0.5)',  
        paper_bgcolor='lightyellow',  
        xaxis_title={
            'text': 'Quarter',
            'font': {'color': 'blue', 'size': 18}  
        },
        yaxis_title={
            'text': 'Transaction',
            'font': {'color': 'green', 'size': 18}  
        },
        font=dict(
            family="Arial, sans-serif",
            size=14,
            color="darkblue"  
        ),
        xaxis=dict(
            tickmode='array',  
            tickvals=[1, 2, 3, 4],  
            ticktext=['1', '2', '3', '4']
        )
    )
    
    st.plotly_chart(fig_line, use_container_width=True)
    return


def yearly_map_trans_district_data(states, district):
    query = """
        SELECT states, district, year,
               sum(transaction_count) as "Transaction count",
               sum(transaction_amount) as "Transaction amount"
               
        FROM df_map_trans
        WHERE states = %s  AND district = %s
		group by (states, year, district)
    """
    
    cursor.execute(query, (states, district))  
    result = cursor.fetchall()

    df = pd.DataFrame(result, columns=["states", "district", "year", "Transaction Count", "Transaction amount"])

    df_long = df.melt(id_vars=['year'], 
                      value_vars=['Transaction Count', "Transaction amount"], 
                      var_name='Select', 
                      value_name='Value')

    fig_line = px.line(
        df_long,
        x='year',
        y='Value',
        color='Select',
        title=f"Yearly Analysis of {district}",
        markers=True,  
        color_discrete_map={"Transaction Count": "green",
                           "Transaction amount": "blue"}
    )
    
    fig_line.update_layout(
        title={'text': f"Yearly Analysis of {district}", 'font': {'color': 'purple', 'size': 22}, 'x': 0.5},
        plot_bgcolor='rgba(230, 230, 230, 0.5)',  
        paper_bgcolor='lightyellow',  
        xaxis_title={'text': 'Years', 'font': {'color': 'blue', 'size': 18}},  
        yaxis_title={'text': 'Transaction', 'font': {'color': 'green', 'size': 18}},  
        font=dict(family="Arial, sans-serif", size=14, color="darkblue"),
        xaxis=dict(
            tickmode='array',  
            tickvals=[2018, 2019, 2020, 2021, 2022, 2023],  
            ticktext=['2018', '2019', '2020', '2021', '2022', '2023']
        )
    )
    
    st.plotly_chart(fig_line, use_container_width=True)
    return  



### map user

def quarterly_map_users_district_data(states, year, district):
    query = """
        SELECT states, district, year, quarter,
               registeredusers as "Registered users",
               appopens as "App opens"
               
        FROM df_map_user
        WHERE states = %s AND year = %s AND district = %s
    """
    cursor.execute(query, (states, year, district))
    result = cursor.fetchall()

    df = pd.DataFrame(result, columns=["states", "district", "year", "quarter", "Registered users", "App opens"])
    df_long = df.melt(id_vars=['quarter'], 
                      value_vars=['Registered users',  "App opens"], 
                      var_name='Select', 
                      value_name='Value')
    
    fig_line = px.line(
        df_long,
        x='quarter',
        y='Value',
        color='Select',
        title=f"Quarterly analysis of users in {district} {year}",
        markers=True,  
        color_discrete_map={
            "Registered users": "blue",
            "App opens": "green"
        }
    )
    
    fig_line.update_layout(
        title={
            'text': f"Quarterly analysis of users in {district} {year}",  
            'font': {'color': 'purple', 'size': 22},  
            'x': 0.5  
        },
        plot_bgcolor='rgba(230, 230, 230, 0.5)',  
        paper_bgcolor='lightyellow',  
        xaxis_title={
            'text': 'Quarter',
            'font': {'color': 'blue', 'size': 18}  
        },
        yaxis_title={
            'text': 'Users',
            'font': {'color': 'green', 'size': 18}  
        },
        font=dict(
            family="Arial, sans-serif",
            size=14,
            color="darkblue"  
        ),
        xaxis=dict(
            tickmode='array',  
            tickvals=[1, 2, 3, 4],  
            ticktext=['1', '2', '3', '4']
        )
    )
    
    st.plotly_chart(fig_line, use_container_width=True)
    return

def yearly_map_user_district_data(states, district):
    query = """
        SELECT states, district, year,
              sum(registeredusers) as "Registered users",
                sum(appopens) as "App opens"
               
        FROM df_map_user
        WHERE states = %s  AND district = %s
		group by (states, year, district)
    """
    
    cursor.execute(query, (states, district))  
    result = cursor.fetchall()

    df = pd.DataFrame(result, columns=["states", "district", "year", "Registered users", "App opens"])
    df_long = df.melt(id_vars=['year'], 
                      value_vars=['Registered users', "App opens"], 
                      var_name='Select', 
                      value_name='Value')

    fig_line = px.line(
        df_long,
        x='year',
        y='Value',
        color='Select',
        title=f"Yearly Analysis of users in {district}",
        markers=True,  
        color_discrete_map={"Registered users": "green",
                           "App opens": "blue"}
    )
    
    fig_line.update_layout(
        title={'text': f"Yearly Analysis of users in {district}", 'font': {'color': 'purple', 'size': 22}, 'x': 0.5},
        plot_bgcolor='rgba(230, 230, 230, 0.5)',  
        paper_bgcolor='lightyellow',  
        xaxis_title={'text': 'Years', 'font': {'color': 'blue', 'size': 18}},  
        yaxis_title={'text': 'users', 'font': {'color': 'green', 'size': 18}},  
        font=dict(family="Arial, sans-serif", size=14, color="darkblue"),
        xaxis=dict(
            tickmode='array',  
            tickvals=[2018, 2019, 2020, 2021, 2022, 2023],  
            ticktext=['2018', '2019', '2020', '2021', '2022', '2023']
        )
    )
    
    st.plotly_chart(fig_line, use_container_width=True)
    return   

### streamlit



st.set_page_config(layout="wide")

def get_base64(file_path):
    with open(file_path, "rb") as f:
        return base64.b64encode(f.read()).decode()

image_path = "C:/Users/RAJ/Downloads/phonepe.jpg"  
base64_image = get_base64(image_path)

if "page" not in st.session_state:
    st.session_state.page = "HOME"
if "data_analysis_option" not in st.session_state:
    st.session_state.data_analysis_option = "Transaction"

col1, col2, col3, col4 = st.columns(4, gap="large")
with col1:
    if st.button("HOME", use_container_width=True):
        st.session_state.page = "HOME"
with col2:
    if st.button("DATA ANALYSIS", use_container_width=True):
        st.session_state.page = "DATA ANALYSIS"
with col3:
    if st.button("TIME SERIES ANALYSIS", use_container_width=True):
        st.session_state.page = "TIME SERIES ANALYSIS"
with col4:
    if st.button("INSIGHTS", use_container_width=True):
        st.session_state.page = "INSIGHTS"

if st.session_state.page == "HOME":
    st.markdown(
        f"""
        <style>
        .stApp {{
            background-image: url("data:image/jpg;base64,{base64_image}");
            background-size: cover;
            background-position: center;
            background-attachment: fixed;
        }}
        .center-text {{
            text-align: center;
            font-size: 36px;
            font-weight: bold;
        }}
        .dashboard-text {{
            text-align: center;
            font-size: 20px;
            max-width: 800px;
            margin: auto;
            padding: 20px;
            background: rgba(255, 255, 255, 0.8);
            border-radius: 10px;
        }}
        </style>
        """,
        unsafe_allow_html=True
    )

    st.markdown(
    """
    <h3 style='font-size:30px; color: red;'>About this Dashboard</h3>
    <p style='font-size:22px; color: red;'>
        This interactive dashboard provides a <b>comprehensive analysis</b> of PhonePe transactions across India.
    </p>
    <p style='font-size:22px; color: red;'>
        It allows users to <b>explore trends, visualize data, and gain insights</b> into digital payments.
    </p>
    """,
    unsafe_allow_html=True
    )


elif st.session_state.page == "DATA ANALYSIS":
    analysis_type = st.selectbox("Select", ["Transaction", "User"])
    st.session_state.data_analysis_option = analysis_type

    if analysis_type == "Transaction":

        cursor.execute("SELECT DISTINCT year FROM aggregated_transaction ORDER BY year;")
        years_a_t = [row[0] for row in cursor.fetchall()]
        cursor.execute("SELECT DISTINCT quarter FROM aggregated_transaction ORDER BY quarter;")
        quarters_a_t = [row[0] for row in cursor.fetchall()]
        cursor.execute("SELECT DISTINCT states FROM aggregated_transaction ORDER BY states;")
        states_a_t = [row[0] for row in cursor.fetchall()]
        

        st.markdown("""<h2><b>ANALYSIS BY ALL STATES ON INDIA MAP</b></h2>""", unsafe_allow_html=True)
        col1, col2, col3 = st.columns([1, 1, 2])
        with col1:
            selected_year = st.selectbox("Select Year", options=["All"] + years_a_t, index=0)
            selected_year = None if selected_year == "All" else selected_year
        with col2:
            selected_quarter = st.selectbox("Select Quarter", options=[None] + quarters_a_t, index=0)
        with col3:
            selected_column = st.radio("Select Column", ["Transaction Count", "Transaction Amount"], horizontal=True)
        aggregated_transaction_map_with_allstate_year_quarter(cursor, selected_year, selected_quarter, selected_column)

        
        st.markdown("""<h2><b>BAR PLOT ANALYSIS BY ALL STATES.</b></h2>""", unsafe_allow_html=True)
        col4, col5 = st.columns([1, 1])
        with col4:
            selected_year1 = st.selectbox("Select Year", options=["All"] + years_a_t, index=0, key="year_2")  
            selected_year1 = None if selected_year1 == "All" else selected_year1
        with col5:
            selected_quarter1 = st.selectbox("Select Quarter", options=[None] + quarters_a_t, index=0, key="quarter_2")  
        aggregated_transaction_with_state_year_quarter(cursor,selected_year1, selected_quarter1)


        st.markdown("""<h2><b>TRANSACTION TYPE ANALYSIS ON A SELECTED STATE OR ALL STATES</b></h2>""", unsafe_allow_html=True)
        col6, col7, col8 = st.columns([1, 1, 2])
        with col6:
            selected_year3 = st.selectbox("Select Year", options=["All"] + years_a_t, index=0, key="year_3")
            selected_year3 = None if selected_year3 == "All" else selected_year3
        with col7:
            selected_quarter3 = st.selectbox("Select Quarter", options=[None] + quarters_a_t, index=0, key="quarter_3")
        with col8:
            selected_states3 = st.selectbox("Select states", options=["All"] + states_a_t, index=0, key="states_3")
            selected_states3= None if selected_states3 == "All" else selected_states3
        aggregated_transaction_transaction_type_with_state_year_quarter(cursor,selected_year3, selected_states3, selected_quarter3)


        cursor.execute("SELECT DISTINCT year FROM df_map_trans ORDER BY year;")
        years_m_t = [row[0] for row in cursor.fetchall()]
        cursor.execute("SELECT DISTINCT quarter FROM df_map_trans ORDER BY quarter;")
        quarters_m_t = [row[0] for row in cursor.fetchall()]
        cursor.execute("SELECT DISTINCT states FROM df_map_trans ORDER BY states;")
        states_m_t = [row[0] for row in cursor.fetchall()]
        
        st.markdown("""<h2><b>DISTRICTS TRANSACTION ANALYSIS</b></h2>""", unsafe_allow_html=True)
        col16, col17, col18 = st.columns([1, 1, 2])
        with col16:
            selected_year7 = st.selectbox("Select Year", options=["All"] + years_m_t, index=0, key="year_7")
            selected_year7 = None if selected_year7 == "All" else selected_year7
        with col17:
            selected_quarter7 = st.selectbox("Select Quarter", options=[None] + quarters_m_t, index=0, key="quarter_7")
        with col18:
            selected_states7 = st.selectbox("Select states", options=[None] + states_m_t, index=0, key="states_7")
        if selected_states7 is None:
            st.write("Select any State")
        else:
            map_transaction_district_with_state_year_quarter(cursor, year=selected_year7, states=selected_states7, quarter=selected_quarter7)



        cursor.execute("SELECT DISTINCT year FROM df_top_trans ORDER BY year;")
        years_t_t = [row[0] for row in cursor.fetchall()]
        cursor.execute("SELECT DISTINCT quarter FROM df_top_trans ORDER BY quarter;")
        quarters_t_t = [row[0] for row in cursor.fetchall()]
        cursor.execute("SELECT DISTINCT states FROM df_top_trans ORDER BY states;")
        states_t_t = [row[0] for row in cursor.fetchall()]
        
        st.markdown("""<h2><b>PINCODE TRANSACTION TABLE</b></h2>""", unsafe_allow_html=True)
        col161, col171, col181 = st.columns([1, 1, 2])
        with col161:
            selected_year8 = st.selectbox("Select Year", options=["All"] + years_t_t, index=0, key="year_8")
            selected_year8 = None if selected_year8 == "All" else selected_year8
        with col171:
            selected_quarter8 = st.selectbox("Select Quarter", options=[None] + quarters_t_t, index=0, key="quarter_8")
        with col181:
            selected_states8 = st.selectbox("Select states", options=[None] + states_t_t, index=0, key="states_8")
        if selected_states8 is None:
            st.write("Select any State")
        else:
            top_transaction_pincodes_with_state_year_quarter(cursor, year=selected_year8, states=selected_states8, quarter=selected_quarter8)        

        

    elif analysis_type == "User":

        cursor.execute("SELECT DISTINCT year FROM df_map_user ORDER BY year;")
        years_m_u = [row[0] for row in cursor.fetchall()]
        cursor.execute("SELECT DISTINCT quarter FROM df_map_user ORDER BY quarter;")
        quarters_m_u = [row[0] for row in cursor.fetchall()]
        cursor.execute("SELECT DISTINCT states FROM df_map_user ORDER BY states;")
        states_m_u = [row[0] for row in cursor.fetchall()]
        
        st.markdown("""<h2><b>USER ANALYSIS ON ALL STATES WITH INDIA MAP</b></h2>""", unsafe_allow_html=True)
        col11, col12, col131 = st.columns([1, 1, 2])
        with col11:
            selected_year5 = st.selectbox("Select Year", options=["All"] + years_m_u, index=0, key="year_5")
            selected_year5 = None if selected_year5 == "All" else selected_year5
        with col12:
            selected_quarter5 = st.selectbox("Select Quarter", options=[None] + quarters_m_u, index=0, key="quarter_5")
        with col131:
            selected_column = st.radio("Select Column", ["Registered users", "App opens"], horizontal=True)
        map_user_map_with_allstate_year_quarter(cursor, year=selected_year5, quarter=selected_quarter5, column = selected_column)
        
        st.markdown("""<h2><b>USER ANALYSIS ON ALL STATES ON BAR PLOT</b></h2>""", unsafe_allow_html=True)
        col9, col10 = st.columns([1, 1])
        with col9:
            selected_year4 = st.selectbox("Select Year", options=["All"] + years_m_u, index=0, key="year_4")
            selected_year4 = None if selected_year4 == "All" else selected_year4
        with col10:
            selected_quarter4 = st.selectbox("Select Quarter", options=[None] + quarters_m_u, index=0, key="quarter_4")
        map_user_with_state_year_quarter(cursor, year=selected_year4, quarter=selected_quarter4)

        cursor.execute("SELECT DISTINCT year FROM aggrigated_user ORDER BY year;")
        years_a_u = [row[0] for row in cursor.fetchall()]
        cursor.execute("SELECT DISTINCT quarter FROM aggrigated_user ORDER BY quarter;")
        quarters_a_u = [row[0] for row in cursor.fetchall()]
        cursor.execute("SELECT DISTINCT states FROM aggrigated_user ORDER BY states;")
        states_a_u = [row[0] for row in cursor.fetchall()]
        
        st.markdown("""<h2><b>BRANDS ANALYSIS </b></h2>""", unsafe_allow_html=True)
        col13, col14, col15 = st.columns([1, 1, 2])
        with col13:
            selected_year6 = st.selectbox("Select Year", options=["All"] + years_a_u, index=0, key="year_6")
            selected_year6 = None if selected_year6 == "All" else selected_year6
        with col14:
            selected_quarter6 = st.selectbox("Select Quarter", options=[None] + quarters_a_u, index=0, key="quarter_6")
        with col15:
            selected_states6 = st.selectbox("Select states", options=["All"] + states_a_u, index=0, key="states_6")
            selected_states6= None if selected_states6 == "All" else selected_states6        
        aggregated_user_brands_state_year_quarter(cursor, year=selected_year6,state =selected_states6, quarter=selected_quarter6)

        cursor.execute("SELECT DISTINCT year FROM df_top_user ORDER BY year;")
        years_t_u = [row[0] for row in cursor.fetchall()]
        cursor.execute("SELECT DISTINCT quarter FROM df_top_user ORDER BY quarter;")
        quarters_t_u = [row[0] for row in cursor.fetchall()]
        cursor.execute("SELECT DISTINCT states FROM df_top_user ORDER BY states;")
        states_t_u = [row[0] for row in cursor.fetchall()]

        st.markdown("""<h2><b>PINCODE USERS ANALYSIS </b></h2>""", unsafe_allow_html=True)
        col23, col24, col25 = st.columns([1, 1, 2])
        with col23:
            selected_year9 = st.selectbox("Select Year", options=["All"] + years_t_u, index=0, key="year_9")
            selected_year9 = None if selected_year9 == "All" else selected_year6
        with col24:
            selected_quarter9 = st.selectbox("Select Quarter", options=[None] + quarters_t_u, index=0, key="quarter_9")
        with col25:
            selected_states9 = st.selectbox("Select states", options=[None] + states_t_u, index=0, key="states_9")
        if selected_states9 is None:
            st.write("Select any State")
        else:
            top_user_pincodes_with_state_year_quarter(cursor, year=selected_year9, states=selected_states9, quarter=selected_quarter9)        

elif st.session_state.page == "TIME SERIES ANALYSIS":
    analysis_type = st.selectbox("Select", ["Transaction", "User"])
    st.session_state.data_analysis_option = analysis_type

    if analysis_type == "Transaction":
        cursor.execute("SELECT DISTINCT year FROM aggregated_transaction ORDER BY year;")
        years_a_t = [row[0] for row in cursor.fetchall()]
        cursor.execute("SELECT DISTINCT quarter FROM aggregated_transaction ORDER BY quarter;")
        quarters_a_t = [row[0] for row in cursor.fetchall()]
        cursor.execute("SELECT DISTINCT states FROM aggregated_transaction ORDER BY states;")
        states_a_t = [row[0] for row in cursor.fetchall()]
        

        st.markdown("""<h2><b> TIME SERIES ANALYSIS ON A STATE IN A YEAR</b></h2>""", unsafe_allow_html=True)
        col1, col2 = st.columns([1, 1])
        with col1:
            selected_year = st.selectbox("Select Year", options= years_a_t, index=0)
        with col2:
            selected_states = st.selectbox("Select State", options= states_a_t, index=0)
        quarterly_transaction_data(selected_year, selected_states)

        col3, _ = st.columns([1, 3])
        with col3:
            selected_states1 = st.selectbox("Select State", options= states_a_t, index=0,  key="states_1")
        yearly_transaction_data(selected_states1)

        cursor.execute("SELECT DISTINCT year FROM df_map_trans ORDER BY year;")
        years_m_t = [row[0] for row in cursor.fetchall()]
        cursor.execute("SELECT DISTINCT quarter FROM df_map_trans ORDER BY quarter;")
        quarters_m_t = [row[0] for row in cursor.fetchall()]
        cursor.execute("SELECT DISTINCT states FROM df_map_trans ORDER BY states;")
        states_m_t = [row[0] for row in cursor.fetchall()]
        


        st.markdown("""<h2><b> TIME SERIES ANALYSIS ON A DISTRICT IN A YEAR</b></h2>""", unsafe_allow_html=True)
        col1mt, col2mt, col3mt = st.columns([1, 1, 2])
        with col1mt:
            selected_year_mt = st.selectbox("Select Year", options= years_m_t, index=0, key="years_1mt")
        with col2mt:
            selected_states_mt = st.selectbox("Select State", options= states_m_t, index=0, key="states_1mt")
            cursor.execute("SELECT DISTINCT district FROM df_map_trans WHERE states = %s", (selected_states_mt,))
            district_m_t = [row[0] for row in cursor.fetchall()]
        with col3mt:
            selected_district_mt = st.selectbox("Select District", options= district_m_t, index=0, key="district_1mt")
        quarterly_map_trans_district_data(selected_states_mt, selected_year_mt, selected_district_mt)


        st.markdown("""<h2><b> TIME SERIES ANALYSIS ON A DISTRICT OVER ALL YEARS</b></h2>""", unsafe_allow_html=True)
        col4mt, col5mt = st.columns([1, 1])
        with col4mt:
            selected_states_2mt = st.selectbox("Select State", options= states_m_t, index=0, key="states_2mt")
            cursor.execute("SELECT DISTINCT district FROM df_map_trans WHERE states = %s", (selected_states_2mt,))
            district2_m_t = [row[0] for row in cursor.fetchall()]
        with col5mt:
            selected_district2_mt = st.selectbox("Select District", options= district2_m_t, index=0, key="district_2mt")
        yearly_map_trans_district_data(selected_states_2mt, selected_district2_mt)

    elif analysis_type == "User":
        cursor.execute("SELECT DISTINCT year FROM aggrigated_user ORDER BY year;")
        years_a_u = [row[0] for row in cursor.fetchall()]
        cursor.execute("SELECT DISTINCT quarter FROM aggrigated_user ORDER BY quarter;")
        quarters_a_u = [row[0] for row in cursor.fetchall()]
        cursor.execute("SELECT DISTINCT states FROM aggrigated_user ORDER BY states;")
        states_a_u = [row[0] for row in cursor.fetchall()]
        cursor.execute("SELECT DISTINCT brands FROM aggrigated_user ORDER BY brands;")
        brands_a_u = [row[0] for row in cursor.fetchall()]

        st.markdown("""<h2><b> TIME SERIES ANALYSIS ON A BRAND IN A YEAR</b></h2>""", unsafe_allow_html=True)
        col1B, col2B = st.columns([1, 1])
        with col1B:
            selected_year_B = st.selectbox("Select Year", options= years_a_u, index=0, key="states_B")
        with col2B:
            selected_brand_B = st.selectbox("Select Brand", options= brands_a_u, index=0, key="brands_B")
        quarterly_agg_user_data(selected_year_B, selected_brand_B)

        st.markdown("""<h2><b> TIME SERIES ANALYSIS ON A BRAND OVER ALL YEARS</b></h2>""", unsafe_allow_html=True)
        col2B, _ = st.columns([1, 3])
        with col2B:
            selected_brand_B1 = st.selectbox("Select Brand", options= brands_a_u, index=0, key="brands_B1")
        yearly_aggrigated_user_data(selected_brand_B1)


        cursor.execute("SELECT DISTINCT year FROM df_map_user ORDER BY year;")
        years_m_u = [row[0] for row in cursor.fetchall()]
        cursor.execute("SELECT DISTINCT quarter FROM df_map_user ORDER BY quarter;")
        quarters_m_u = [row[0] for row in cursor.fetchall()]
        cursor.execute("SELECT DISTINCT states FROM df_map_user ORDER BY states;")
        states_m_u = [row[0] for row in cursor.fetchall()]


        st.markdown("""<h2><b> TIME SERIES ANALYSIS ON A DISTRICT IN A YEAR</b></h2>""", unsafe_allow_html=True)
        col1mu, col2mu, col3mu = st.columns([1, 1, 2])
        with col1mu:
            selected_year_mu = st.selectbox("Select Year", options= years_m_u, index=0, key="years_1mu")
        with col2mu:
            selected_states_mu = st.selectbox("Select State", options= states_m_u, index=0, key="states_1mu")
            cursor.execute("SELECT DISTINCT district FROM df_map_user WHERE states = %s", (selected_states_mu,))
            district_m_u = [row[0] for row in cursor.fetchall()]
        with col3mu:
            selected_district_mu = st.selectbox("Select District", options= district_m_u, index=0, key="district_1mu")
        quarterly_map_users_district_data(selected_states_mu, selected_year_mu, selected_district_mu)


        st.markdown("""<h2><b> TIME SERIES ANALYSIS ON A DISTRICT OVER ALL YEARS</b></h2>""", unsafe_allow_html=True)
        col4mu, col5mu = st.columns([1, 1])
        with col4mu:
            selected_states2_mu = st.selectbox("Select State", options= states_m_u, index=0, key="states_2mu")
            cursor.execute("SELECT DISTINCT district FROM df_map_user WHERE states = %s", (selected_states2_mu,))
            district2_m_u = [row[0] for row in cursor.fetchall()]
        with col5mu:
            selected_district2_mu = st.selectbox("Select District", options= district2_m_u, index=0, key="district_2mu")
        yearly_map_user_district_data(selected_states2_mu, selected_district2_mu)
           

elif st.session_state.page == "INSIGHTS":
    mydb = psycopg2.connect(
    host='localhost',
    user='postgres',
    port='5432',
    database='phonepe',
    password='Liverpool08#'
    )
    cursor = mydb.cursor()
    
    def fetch_data(query):
        cursor.execute(query)
        data = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]  # Get column names
        return pd.DataFrame(data, columns=columns)
    
    st.title("📊 PhonePe Data Analysis Dashboard")

    
    selected_option = st.sidebar.radio("Select an Insight", [
        "Top & Bottom 5 States by Transaction Count",
        "Top & Bottom 5 States by Transaction Amount",
        "Transaction Trends: P2P vs Merchant Payments",
        "Smartphone Market Share Distribution",
        "Top & Bottom 5 Districts by Transaction Count",
        "Top & Bottom 5 Districts by Transaction Amount",
        "Top & Bottom 5 Districts by Registered Users",
        "Average Transaction Count per State",
        "Average Transaction Amount per State",
        "Declining States in Digital Transactions"
    ])
    
    if selected_option == "Top & Bottom 5 States by Transaction Count":
        query_top = """SELECT states, SUM(transaction_count) AS Transaction_count
                       FROM aggregated_transaction 
                       GROUP BY states 
                       ORDER BY Transaction_count DESC LIMIT 5;"""
        query_bottom = """SELECT states, SUM(transaction_count) AS Transaction_count
                          FROM aggregated_transaction 
                          GROUP BY states 
                          ORDER BY Transaction_count ASC LIMIT 5;"""
        
        df_top = fetch_data(query_top)
        df_bottom = fetch_data(query_bottom)
    
        st.subheader("Top 5 States by Transaction Count")
        st.dataframe(df_top)
        st.subheader("Bottom 5 States by Transaction Count")
        st.dataframe(df_bottom)
    
    elif selected_option == "Top & Bottom 5 States by Transaction Amount":
        query_top = """SELECT states, SUM(transaction_amount) AS Transaction_amount
                       FROM aggregated_transaction 
                       GROUP BY states 
                       ORDER BY Transaction_amount DESC LIMIT 5;"""
        query_bottom = """SELECT states, SUM(transaction_amount) AS Transaction_amount
                          FROM aggregated_transaction 
                          GROUP BY states 
                          ORDER BY Transaction_amount ASC LIMIT 5;"""
        
        df_top = fetch_data(query_top)
        df_bottom = fetch_data(query_bottom)
    
        st.subheader("Top 5 States by Transaction Amount")
        st.dataframe(df_top)
        st.subheader("Bottom 5 States by Transaction Amount")
        st.dataframe(df_bottom)
    
    elif selected_option == "Transaction Trends: P2P vs Merchant Payments":
        query_count = """SELECT transaction_type, SUM(transaction_count) AS Transaction_count
                         FROM aggregated_transaction 
                         GROUP BY transaction_type 
                         ORDER BY Transaction_count DESC;"""
        query_amount = """SELECT transaction_type, SUM(transaction_amount) AS Transaction_amount
                          FROM aggregated_transaction 
                          GROUP BY transaction_type 
                          ORDER BY Transaction_amount DESC;"""
    
        df_count = fetch_data(query_count)
        df_amount = fetch_data(query_amount)
    
        st.subheader("Transaction Count by Type")
        st.dataframe(df_count)
        st.subheader("Transaction Amount by Type")
        st.dataframe(df_amount)
    
    elif selected_option == "Smartphone Market Share Distribution":
        st.subheader("Xiaomi Leads the Market Share")
        st.write("""
        - **Xiaomi**: 26.1% (Most Popular)  
        - **Samsung**: 19.2%  
        - **Vivo**: 16.6%  
        - **Oppo**: 11.6%  
        """)
    
    elif selected_option == "Top & Bottom 5 Districts by Transaction Count":
        query_top = """SELECT district, SUM(transaction_count) AS Transaction_count
                       FROM df_map_trans 
                       GROUP BY district 
                       ORDER BY Transaction_count DESC LIMIT 5;"""
        query_bottom = """SELECT district, SUM(transaction_count) AS Transaction_count
                          FROM df_map_trans 
                          GROUP BY district 
                          ORDER BY Transaction_count ASC LIMIT 5;"""
    
        df_top = fetch_data(query_top)
        df_bottom = fetch_data(query_bottom)
    
        st.subheader("Top 5 Districts by Transaction Count")
        st.dataframe(df_top)
        st.subheader("Bottom 5 Districts by Transaction Count")
        st.dataframe(df_bottom)
    
    elif selected_option == "Top & Bottom 5 Districts by Transaction Amount":
        query_top = """SELECT district, SUM(transaction_amount) AS Transaction_amount
                       FROM df_map_trans 
                       GROUP BY district 
                       ORDER BY Transaction_amount DESC LIMIT 5;"""
        query_bottom = """SELECT district, SUM(transaction_amount) AS Transaction_amount
                          FROM df_map_trans 
                          GROUP BY district 
                          ORDER BY Transaction_amount ASC LIMIT 5;"""
    
        df_top = fetch_data(query_top)
        df_bottom = fetch_data(query_bottom)
    
        st.subheader("Top 5 Districts by Transaction Amount")
        st.dataframe(df_top)
        st.subheader("Bottom 5 Districts by Transaction Amount")
        st.dataframe(df_bottom)
    
    elif selected_option == "Top & Bottom 5 Districts by Registered Users":
        query_top = """SELECT district, SUM(registeredusers) AS Registeredusers
                       FROM df_map_user 
                       GROUP BY district 
                       ORDER BY Registeredusers DESC LIMIT 5;"""
        query_bottom = """SELECT district, SUM(registeredusers) AS Registeredusers
                          FROM df_map_user 
                          GROUP BY district 
                          ORDER BY Registeredusers ASC LIMIT 5;"""
    
        df_top = fetch_data(query_top)
        df_bottom = fetch_data(query_bottom)
    
        st.subheader("Top 5 Districts by Registered Users")
        st.dataframe(df_top)
        st.subheader("Bottom 5 Districts by Registered Users")
        st.dataframe(df_bottom)
    
    elif selected_option == "Average Transaction Count per State":
        query = """SELECT states, AVG(transaction_count) AS Transaction_count
                   FROM aggregated_transaction 
                   GROUP BY states 
                   ORDER BY Transaction_count DESC;"""
    
        df = fetch_data(query)
        st.subheader("Average Transaction Count per State")
        st.dataframe(df)
    
    elif selected_option == "Average Transaction Amount per State":
        query = """SELECT states, AVG(transaction_amount) AS Transaction_amount
                   FROM aggregated_transaction 
                   GROUP BY states 
                   ORDER BY Transaction_amount DESC;"""
    
        df = fetch_data(query)
        st.subheader("Average Transaction Amount per State")
        st.dataframe(df)
    
    elif selected_option == "Declining States in Digital Transactions":
        st.subheader("States Showing Declining Trends in Transactions")
        st.write("""
        - **Chandigarh**  
        - **Manipur**  
        These states have shown a decline in both transaction count and amount since 2022.
        """)
    
